In [59]:
# import the necessary packages
import numpy as np
import argparse
import imutils
import sys
import cv2
import matplotlib.pyplot as plt
import os
from collections import Counter

In [35]:
model = "./resnet-34_kinetics.onnx"

In [36]:
net = cv2.dnn.readNet(model)

In [37]:
from os import listdir
from os.path import isfile, join

mypath = "./kinetics400/kinetics-downloader/dataset/test/"

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [43]:
len(onlyfiles)

122

In [38]:
import pandas as pd

In [39]:
csv_path = "./kinetics400/test.csv"
labels = pd.read_csv(csv_path)

In [40]:
labels = labels.drop('time_start', 1)
labels = labels.drop('time_end', 1)
labels = labels.drop('split', 1)

/tmp/ipykernel_6183/45598693.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  labels = labels.drop('time_start', 1)
/tmp/ipykernel_6183/45598693.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  labels = labels.drop('time_end', 1)
/tmp/ipykernel_6183/45598693.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  labels = labels.drop('split', 1)


In [41]:
labels = labels.set_index('youtube_id')
labels

,label
youtube_id,
--6bJUbfpnQ,drinking beer
--8YXc8iCt8,climbing tree
--coBvtS-eQ,surfing water
--q6ElFyVq0,stomping grapes
--q_mvQ8zP8,tai chi
...,...
zzHsdlYe_5I,catching or throwing softball
zzl-3zkieiE,skiing (not slalom or crosscountry)
zzpqbqLllzA,jumping into pool


In [42]:
d = labels.to_dict('index')
print(d['--6bJUbfpnQ']['label'])

drinking beer


In [47]:
SAMPLE_DURATION = 16 
SAMPLE_SIZE = 112 

names = "./action_recognition_kinetics.txt"
with open(names) as l:
    CLASSES = l.read().strip().split("\n")

In [63]:
for video in onlyfiles:
    v_path = os.path.join(mypath, video)
    #print(v_path)
    vs = cv2.VideoCapture(v_path)
    # initialize the list / dictionaries to captured classified frames
    classifiedFrames = []
    framesDict = []
    predictions = []
    dictKey = {}
    correct = 0
    count = 0
    n = 0
    
    # Run the model
    # loop until we explicitly break from it
    while True:
        # initialize the batch of frames that will be passed through the model
        frames = []
        n += 1
        end_of_video=False
    
        # loop over the number of required sample frames
        for i in range(16):
            # read a frame from the video stream
            (grabbed, frame) = vs.read()
            #Identiy the frame number
            pos_frame = vs.get(cv2.CAP_PROP_POS_FRAMES)
    
            # if the frame was not grabbed then we've reached the end of
            # the video stream so exit the script
            if not grabbed:
                end_of_video=True
                break
            # otherwise, the frame was read so resize it and add it to
            # our frames list
            frame = imutils.resize(frame, width=400)
            frames.append(frame)
  
        # now that our frames array is filled we can construct our blob
        if not end_of_video:
            blob = cv2.dnn.blobFromImages(frames, 1.0, (SAMPLE_SIZE, SAMPLE_SIZE), (114.7748, 107.7354, 99.4750), 
                                swapRB=True, crop=True)
            blob = np.transpose(blob, (1, 0, 2, 3))
            blob = np.expand_dims(blob, axis=0)
  
            # pass the blob through the network to obtain our human activity
            # recognition predictions
            net.setInput(blob)
            outputs = net.forward()
            prediction = CLASSES[np.argmax(outputs)]
            predictions.append(prediction)
        
        else:
            major_pred, conf = Counter(predictions).most_common()[0]
            break
    answer = d[video.split(".")[0]]['label']
    print(answer)
    print(major_pred)
    count += 1
    if answer==major_pred:
        correct += 1

print("Acc:" + str(float(correct/count)))

cartwheeling
cartwheeling
climbing tree
climbing tree
shredding paper
shredding paper
shoveling snow
shoveling snow
knitting
knitting
bartending
bartending
snorkeling
snorkeling
flipping pancake
flipping pancake
ironing
ironing
climbing a rope
disc golfing
playing piano
playing piano
playing guitar
playing cello
juggling fire
spinning poi
krumping
breakdancing
springboard diving
springboard diving
making a sandwich
making pizza
playing keyboard
playing keyboard
breakdancing
dribbling basketball
breakdancing
punching person (boxing)
country line dancing
country line dancing
tango dancing
tango dancing
riding camel
riding camel
gymnastics tumbling
gymnastics tumbling
scuba diving
canoeing or kayaking
unloading truck
digging
yoga
cheerleading
ice climbing
ice climbing
playing poker
playing poker
picking fruit
picking fruit
bookbinding
bookbinding
tap dancing
country line dancing
laughing
laughing
washing hair
massaging back
barbequing
barbequing
tying knot (not on a tie)
knitting
skiing s

KeyboardInterrupt: 